# Importing Dependencies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import cv2
import os
import keras
from glob import glob
import tensorflow as tf
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras import backend as K

In [2]:
tf.config.run_functions_eagerly(True)

In [3]:
gpus = tf.config.list_physical_devices('GPU')
for i in gpus:
    i = tf.config.experimental.set_memory_growth(i, True)

In [4]:
label = 5
train_path = './train/'
model_path = os.path.join('./model/', f'unet_jacc_{label}.keras')
log_path = os.path.join('./log/', f'log_{label}.csv')

In [5]:
batch_size = 3
lr = 1e-4
height = 1024//4
width = 1280//4

## Model Functions

In [6]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv2D(num_filters, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    return x

In [7]:
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

In [8]:
def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding='same')(inputs)
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

In [9]:
def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding = 'same', activation='sigmoid')(d4)
    model = Model(inputs, outputs, name='UNET')
    return model

In [10]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (width, height))
    x = x/255.0
    return x

In [11]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (width, height))
    x = x/255.0
    x = np.expand_dims(x, axis=-1)
    return x

In [12]:
def convert_to_binary(image_path : str):
    image = cv2.imread(image_path,0)
    numpy_image = np.array([[255 if pixel == 29 else 0 for pixel in row]for row in image])
    return numpy_image.astype(np.uint8)

In [13]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y
    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([height, width, 3])
    y.set_shape([height, width, 1])

    return x, y

In [14]:
def tf_dataset(x, y, batch=16):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

# Loading Dataset

In [15]:
X = glob(os.path.join(train_path, 'frames', '*'))
y = glob(os.path.join(train_path, f'labels_{label}', '*'))

In [16]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(X, y, train_size=0.8, shuffle=False, random_state=27) 

In [17]:
print(f'Train: {len(train_x)} - {len(train_y)}')
print(f'Valid: {len(val_x)} - {len(val_y)}')

Train: 1788 - 1788
Valid: 447 - 447


In [18]:
train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(val_x, val_y, batch=batch_size)

C:\Users\joshu\AppData\Roaming\Python\Python310\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


# U-NET Model

In [19]:
input_shape = (height, width, 3)
model = build_unet(input_shape)
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 320, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 320, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 320, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                              

In [20]:
def jaccard_distance(y_true, y_pred, smooth=100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

In [21]:
opt = tf.keras.optimizers.Adam(lr)
model.compile(loss = jaccard_distance, optimizer=opt, metrics=[jaccard_distance])

In [22]:
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2),
    CSVLogger(log_path),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
]

In [23]:
model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=10,
    callbacks=callbacks
)

Epoch 1/10
596/596 [==============================] - ETA: 0s - loss: 0.0479 - jaccard_distance: 0.0479
Epoch 1: val_loss improved from inf to 0.02602, saving model to ./model\unet_jacc_5.keras
596/596 [==============================] - 140s 217ms/step - loss: 0.0479 - jaccard_distance: 0.0479 - val_loss: 0.0260 - val_jaccard_distance: 0.0260 - lr: 1.0000e-04
Epoch 2/10
596/596 [==============================] - ETA: 0s - loss: 0.0178 - jaccard_distance: 0.0178
Epoch 2: val_loss improved from 0.02602 to 0.01617, saving model to ./model\unet_jacc_5.keras
596/596 [==============================] - 129s 216ms/step - loss: 0.0178 - jaccard_distance: 0.0178 - val_loss: 0.0162 - val_jaccard_distance: 0.0162 - lr: 1.0000e-04
Epoch 3/10
596/596 [==============================] - ETA: 0s - loss: 0.0111 - jaccard_distance: 0.0111
Epoch 3: val_loss improved from 0.01617 to 0.00954, saving model to ./model\unet_jacc_5.keras
596/596 [==============================] - 129s 216ms/step - loss: 0.0111 

# Export Model

In [24]:
model.save(model_path)

In [25]:
Model = tf.keras.models.load_model(model_path, custom_objects={'jaccard_distance': jaccard_distance})
Model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 320, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 320, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 256, 320, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                              